### Import package

In [119]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

tensorflow.keras.utils.set_random_seed(812)

pd.set_option('future.no_silent_downcasting', True)

### Read in files

In [102]:
def read_protein_file(file_path):
    with open(file_path, "r") as file:
        # Read the entire contents of the file
        seq = ""
        str = ""

        sequences = []
        strings = []
        count = 0
        for line in file:
            if count < 8:
                pass
            count += 1

            line = line.strip()
            if line == "<>":
                seq = ""
                str = ""
            elif line == "<end>" or line=="end":
                sequences.append(seq)
                strings.append(str)
            else:
                letters = line.split(" ")
                if len(letters) == 2:
                    seq += letters[0]
                    str += letters[1] if letters[1] != "_" else "c"
        return sequences, strings


In [103]:
train_seq, train_str = read_protein_file("data/protein-secondary-structure.train")
test_seq, test_str = read_protein_file("data/protein-secondary-structure.test")

### Create sliding windows

In [104]:
def split_based_on_windows(data_seq, data_str, W=17):
    all_sequences = []
    all_strings = []
    for i in range(len(data_seq)):
        sequences = [data_seq[i][j:j+W] for j in range(0, len(data_seq[i]), W)]
        strings = [data_str[i][j:j+W] for j in range(0, len(data_str[i]), W)]

        all_sequences += sequences
        all_strings += strings

    train_df = pd.DataFrame({"sequence": all_sequences, "string": all_strings})
    return train_df


In [105]:
WINDOW_SIZE = 17

In [106]:
train_df = split_based_on_windows(train_seq, train_str, WINDOW_SIZE)
test_df = split_based_on_windows(test_seq, test_str, WINDOW_SIZE)

In [107]:
train_df.head()

,sequence,string
0,GVGTVPMTDYGNDVEYY,ccccccccccccccccc
1,GQVTIGTPGKSFNLNFD,cceecccccceecceee
2,TGSSNLWVGSVQCQASG,cccccceeccccccccc
3,CKGGRDKFNPSDGSTFK,ccccccccccccccccc
4,ATGYDASIGYGDGSASG,cccccccccccccccee


### One hot encode the data

In [108]:
PROTEIN_LETTERS = 'ACDEFGHIKLMNPQRSTVWXY'
SECONDARY_LETTERS = 'ceh'

In [109]:
def ohe_for_nn(sequences, strings=None):
    
    X_ohe = [[PROTEIN_LETTERS.index(letter) for letter in seq] for seq in sequences]
    max_length = max(len(seq) for seq in X_ohe)
    X_padded = pad_sequences(X_ohe, maxlen=max_length, padding='post')
    X = np.zeros((len(X_padded), max_length, len(PROTEIN_LETTERS)))

    if strings:
        y_ohe = [[SECONDARY_LETTERS.index(letter) for letter in string] for string in strings]
        y_padded = pad_sequences(y_ohe, maxlen=max_length, padding='post')
        y = np.zeros((len(y_padded), max_length, len(SECONDARY_LETTERS)))

    for i in range(len(X_padded)):
        for j, aa_index in enumerate(X_padded[i]):
            X[i, j, aa_index] = 1
        if strings:
            for j, structure_index in enumerate(y_padded[i]):
                y[i, j, structure_index] = 1

    if strings:
        return X, y
    else:
        return X

In [110]:
X_train, y_train = ohe_for_nn(train_df['sequence'], train_df['string'])
X_test, y_test = ohe_for_nn(test_df['sequence'], test_df['string'])

### Create the neural network

In [111]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Embedding, Bidirectional
from tensorflow.keras import layers

In [112]:
def create_model():
    drop_out = 0.3

    model = Sequential([
    LSTM(128, input_shape=(WINDOW_SIZE, len(PROTEIN_LETTERS)), return_sequences=True),
    layers.BatchNormalization(),
    layers.Dropout(drop_out),
    Dense(64, activation='tanh'),
    layers.BatchNormalization(),
    layers.Dropout(drop_out),
    Dense(128, activation='linear'),
    layers.BatchNormalization(),
    layers.Dropout(drop_out),
    Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(drop_out),
    Dense(32, activation='relu'),
    TimeDistributed(Dense(len(SECONDARY_LETTERS), activation='softmax')) # Apply Dense layer to each time step
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'mae'])

    return model

In [113]:
# Define the neural network architecture

model = create_model()

# # Model from https://github.com/LucaAngioloni/ProteinSecondaryStructure-CNN
# model = Sequential([
#     layers.Conv1D(128, 5, padding='same', activation='relu', input_shape=(WINDOW_SIZE, len(PROTEIN_LETTERS))),
#     layers.BatchNormalization(),
#     layers.Dropout(drop_out),
#     layers.Conv1D(128, 3, padding='same', activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(drop_out),
#     layers.Conv1D(64, 3, padding='same', activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(drop_out),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(32, activation='relu'),
#     TimeDistributed(Dense(len(SECONDARY_LETTERS), activation='softmax')) # Apply Dense layer to each time step
# ])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
loss, accuracy, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}, Test MAE: {mae}')


Epoch 1/100
24/24 [==============================] - 4s 44ms/step - loss: 1.1835 - accuracy: 0.4633 - mae: 0.3919 - val_loss: 1.0199 - val_accuracy: 0.5580 - val_mae: 0.4234
Epoch 2/100
24/24 [==============================] - 0s 16ms/step - loss: 1.0085 - accuracy: 0.5470 - mae: 0.3552 - val_loss: 0.9907 - val_accuracy: 0.5580 - val_mae: 0.4129
Epoch 3/100
24/24 [==============================] - 0s 17ms/step - loss: 0.9668 - accuracy: 0.5567 - mae: 0.3536 - val_loss: 0.9794 - val_accuracy: 0.5580 - val_mae: 0.4083
Epoch 4/100
24/24 [==============================] - 0s 15ms/step - loss: 0.9490 - accuracy: 0.5662 - mae: 0.3504 - val_loss: 0.9722 - val_accuracy: 0.5580 - val_mae: 0.4054
Epoch 5/100
24/24 [==============================] - 0s 18ms/step - loss: 0.9350 - accuracy: 0.5682 - mae: 0.3523 - val_loss: 0.9595 - val_accuracy: 0.5580 - val_mae: 0.3994
Epoch 6/100
24/24 [==============================] - 0s 17ms/step - loss: 0.9252 - accuracy: 0.5739 - mae: 0.3489 - val_loss: 0.95

In [114]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 7ms/step


### Convert the prediction back

In [115]:
def convert_pred_to_str(predictions):
    inv_structure_map = {0: 'c', 1:'e', 2:'h'}
    y_pred_classes = np.argmax(predictions, axis=-1)
    protein_array = np.vectorize(inv_structure_map.get)(y_pred_classes)

    array_strings = ["".join(map(str, row)) for row in protein_array]
    return array_strings

In [116]:
result = convert_pred_to_str(predictions)

In [117]:
test_df['prediction'] = result

In [118]:
test_df

,sequence,string,prediction
0,ENLKLGFLVKQPEEPWF,cccccceeeccccccch,ccccccccecccccccc
1,QTEWKFADKAGKDLGFE,hhhhhhhhhhhhhcccc,cccchhhhhhccccccc
2,VIKIAVPDGEKTLNAID,cceeeccchhhhhhhhh,cececccccceeeeeec
3,SLAASGAKGFVICTPDP,hhhhccccccccccccc,ccccccccccceecccc
4,KLGSAIVAKARGYDMKV,cccchhhhhhhhhcccc,ccccceehhhhcccccc
...,...,...,...
185,VKLVSWYDNEFGYSERV,eeeecccchhhhhhhhh,ccccceeccccceeeee
186,VDLMAHMASKE,hhhhhhhhhhc,ccchhhhcccccccecc
187,SIPPEVKFNKPFVFLMI,cccceeecccceeeeee,cccccecccccccceee
188,EQNTKSPLFMGKVVNPT,ecccceeeeeeeecccc,ccccccceeeeeeeccc
